# Netflix Stock Price Prediction

Dataset: yfinance

Example Solution: https://thecleverprogrammer.com/2022/02/08/netflix-stock-price-prediction-with-machine-learning/

Hugging Face: https://huggingface.co/spaces/alperugurcan/netflix-stock-prediction

In [1]:
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
data = yf.download('NFLX', start='2019-01-01', end='2024-03-14')['Close'].values.reshape(-1, 1)

[*********************100%***********************]  1 of 1 completed


In [3]:
scaler = MinMaxScaler().fit(data)
scaled_data = scaler.transform(data)

In [4]:
# Create sequences
X, y = [], []
for i in range(len(scaled_data) - 60):
    X.append(scaled_data[i:i+60])
    y.append(scaled_data[i+60])
X, y = np.array(X), np.array(y)

In [5]:
# Split data
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [6]:
# Build and train model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(60, 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

C:\Users\alperugurcan\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Predict and print result
prediction = scaler.inverse_transform(model.predict(X_test))
print(f"Predicted Netflix stock price: ${prediction[-1][0]:.2f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 157ms/step
Predicted Netflix stock price: $608.68


In [8]:
model.save('netflix_stock_prediction_model.keras')
